<a href="https://colab.research.google.com/github/Caswell1083/Tjabadi-Caswell/blob/main/Day%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import pandas as pd

In [34]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

run('pip install --upgrade pip')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')
run('gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/')

>> pip install --upgrade pip

>> pip install --quiet apache-beam

>> mkdir -p data

>> gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/
Copying gs://dataflow-samples/shakespeare/kinglear.txt...
/ [1 files][153.6 KiB/153.6 KiB]                                                
Operation completed over 1 objects/153.6 KiB.                                    



In [35]:
! wc -l data/kinglear.txt

5525 data/kinglear.txt


In [36]:
! head -3 data/kinglear.txt

	KING LEAR




In [37]:
import apache_beam as beam
import re

inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

In [38]:
! head -10 data/kinglear.txt > data/small.txt
! wc -l data/small.txt

10 data/small.txt


In [39]:
def myprint(x):
  print('{}'.format(x))

with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/small.txt')
      | "print" >> beam.Map(myprint)
  )

result = pipeline.run()
result.wait_until_finish()

	KING LEAR


	DRAMATIS PERSONAE


LEAR	king of Britain  (KING LEAR:)

KING OF FRANCE:

	KING LEAR


	DRAMATIS PERSONAE


LEAR	king of Britain  (KING LEAR:)

KING OF FRANCE:



'DONE'

In [40]:
with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/small.txt')
      | 'get words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | "print" >> beam.Map(myprint)
  )

KING
LEAR
DRAMATIS
PERSONAE
LEAR
king
of
Britain
KING
LEAR
KING
OF
FRANCE


In [41]:
def my_line_split_func(line):
  return re.findall(r"[a-zA-Z']+", line)

with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/small.txt')
      | 'get words' >> beam.FlatMap(my_line_split_func)
      | "print" >> beam.Map(myprint)
  )

KING
LEAR
DRAMATIS
PERSONAE
LEAR
king
of
Britain
KING
LEAR
KING
OF
FRANCE


In [42]:
! echo -e 'r1c1,r1c2,2020/03/05\nr2c1,r2c2,2020/03/23' > data/play.csv

In [43]:
class Transform(beam.DoFn):

  # Use classes to perform transformations on your PCollections
  # Yield or return the element(s) needed as input for the next transform
  def process(self, element):
    yield element


with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/play.csv')
      | 'format line' >> beam.ParDo(Transform())
      | "print" >> beam.Map(myprint)
  )


result.wait_until_finish()

r1c1,r1c2,2020/03/05
r2c1,r2c2,2020/03/23


'DONE'

In [44]:
csv_path = '/content/drive/My Drive/users_v.csv'


In [45]:
import apache_beam as beam
from datetime import datetime

# Correct path to your CSV file
csv_path = '/content/drive/My Drive/users_v.csv'

# Apache Beam Pipeline
class SplitRow(beam.DoFn):
    def process(self, row):
        # Split each row into its respective columns
        user_id, name, gender, age, address, date_joined = row.split(',')
        yield {
            'user_id': user_id,
            'name': name,
            'gender': gender,
            'age': age,
            'address': address,
            'date_joined': date_joined
        }

class FormatDateAndAddress(beam.DoFn):
    def process(self, element):
        try:
            # Reformat the date assuming the format is YYYY/MM/DD
            formatted_date = datetime.strptime(element['date_joined'], '%Y/%m/%d').strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error formatting date: {e}")
            formatted_date = element['date_joined']  # Keep original date if formatting fails

        # Reformat the address
        address_parts = element['address'].split(',')

        # Handle cases where the address has fewer parts
        if len(address_parts) >= 3:
            formatted_address = f"{address_parts[0]},{address_parts[1]},{address_parts[2]}"
        elif len(address_parts) == 2:
            formatted_address = f"{address_parts[0]},{address_parts[1]}"
        elif len(address_parts) == 1:
            formatted_address = address_parts[0]
        else:
            formatted_address = "Unknown"  # Fallback for missing address parts

        # Update the element with formatted values
        element['date_joined'] = formatted_date
        element['address'] = formatted_address
        yield element

# Build the Beam pipeline
with beam.Pipeline() as pipeline:
    user_data = (
        pipeline
        | 'Read User Data' >> beam.io.ReadFromText(csv_path, skip_header_lines=1)
        | 'Split Row' >> beam.ParDo(SplitRow())
        | 'Format Date and Address' >> beam.ParDo(FormatDateAndAddress())
        | 'Write to CSV' >> beam.io.WriteToText('/content/formatted_output.csv', file_name_suffix='.csv')
    )




In [46]:
# Build the Beam pipeline
with beam.Pipeline() as pipeline:
    # Task 1: Read and format user data
    user_data = (
        pipeline
        | 'Read User Data' >> beam.io.ReadFromText(csv_path, skip_header_lines=1)
        | 'Split Row' >> beam.ParDo(SplitRow())
        | 'Format Date and Address' >> beam.ParDo(FormatDateAndAddress())
    )

    # Task 2: Perform aggregations

    # 1. Gender composition (% split between male and female)
    gender_composition = (
        user_data
        | 'Count Gender' >> beam.Map(lambda element: (element['gender'], 1))
        | 'CountPerGender' >> beam.combiners.Count.PerKey()
        | 'Write Gender Composition' >> beam.io.WriteToText('/content/gender_composition.csv', file_name_suffix='.csv')
    )

    # 2. Count customers by date joined
    customer_by_date = (
        user_data
        | 'Count By Date' >> beam.Map(lambda element: (element['date_joined'], 1))
        | 'CountPerDate' >> beam.combiners.Count.PerKey()
        | 'Write Date Counts' >> beam.io.WriteToText('/content/customer_by_date.csv', file_name_suffix='.csv')
    )

    # 3. Count customers by state
    customer_by_state = (
        user_data
        | 'Count By State' >> beam.Map(lambda element: (element['address'].split(',')[1], 1) if len(element['address'].split(',')) > 1 else ('Unknown', 1))
        | 'CountPerState' >> beam.combiners.Count.PerKey()
        | 'Write State Counts' >> beam.io.WriteToText('/content/customer_by_state.csv', file_name_suffix='.csv')
    )


In [48]:
'Write Gender Composition' >> beam.io.WriteToText('/content/gender_composition', file_name_suffix='.csv')




<_NamedPTransform(PTransform) label=[Write Gender Composition] at 0x7c29b8fb0c40>

In [49]:
!ls /content


beam-temp-formatted_output.csv-c01963e272d111ef91070242ac1c000c
customer_by_date.csv-00000-of-00001.csv
customer_by_state.csv-00000-of-00001.csv
data
drive
formatted_output.csv-00000-of-00001.csv
gender_composition.csv-00000-of-00001.csv
sample_data


In [50]:
pipeline.run().wait_until_finish()


'DONE'

In [53]:
# Build the Beam pipeline
with beam.Pipeline() as pipeline:
    user_data = (
        pipeline
        | 'Read User Data' >> beam.io.ReadFromText(csv_path, skip_header_lines=1)
        | 'Split Row' >> beam.ParDo(SplitRow())
        | 'Format Date and Address' >> beam.ParDo(FormatDateAndAddress())
    )

    # Task 2: Perform aggregations

    # 1. Gender composition (% split between male and female)
    gender_composition = (
        user_data
        | 'Count Gender' >> beam.Map(lambda element: (element['gender'], 1))
        | 'CountPerGender' >> beam.combiners.Count.PerKey()
        | 'Write Gender Composition' >> beam.io.WriteToText('/content/gender_composition.csv', shard_name_template='')
    )

    # 2. Count customers by date joined
    customer_by_date = (
        user_data
        | 'Count By Date' >> beam.Map(lambda element: (element['date_joined'], 1))
        | 'CountPerDate' >> beam.combiners.Count.PerKey()
        | 'Write Date Counts' >> beam.io.WriteToText('/content/customer_by_date.csv', shard_name_template='')
    )

    # 3. Count customers by state
    customer_by_state = (
        user_data
        | 'Count By State' >> beam.Map(lambda element: (element['address'].split(',')[1], 1) if len(element['address'].split(',')) > 1 else ('Unknown', 1))
        | 'CountPerState' >> beam.combiners.Count.PerKey()
        | 'Write State Counts' >> beam.io.WriteToText('/content/customer_by_state.csv', shard_name_template='')
    )

pipeline.run().wait_until_finish()  # Ensure the pipeline finishes execution



'DONE'

In [54]:
!ls /content


beam-temp-formatted_output.csv-c01963e272d111ef91070242ac1c000c
customer_by_date.csv
customer_by_date.csv-00000-of-00001.csv
customer_by_state.csv
customer_by_state.csv-00000-of-00001.csv
data
drive
formatted_output.csv-00000-of-00001.csv
gender_composition.csv
gender_composition.csv-00000-of-00001.csv
sample_data


In [55]:
!cat /content/gender_composition.csv-00000-of-00001.csv


('male', 1207)
('female', 1150)


In [56]:
!cat /content/customer_by_date.csv-00000-of-00001.csv


('2019-03-13', 1)
('2020-11-06', 1)
('2004-05-29', 2)
('2005-08-26', 1)
('2018-04-30', 1)
('2007-05-25', 1)
('2005-01-05', 2)
('2003-12-12', 2)
('2015-11-14', 1)
('2003-05-15', 1)
('2003-10-15', 2)
('2013-09-27', 2)
('2002-03-13', 1)
('2020-12-26', 3)
('2015-11-13', 1)
('2017-07-12', 1)
('2005-02-23', 2)
('2008-08-09', 1)
('2000-08-09', 1)
('2014-05-18', 2)
('2002-02-22', 1)
('2006-04-11', 1)
('2003-03-09', 1)
('2019-03-07', 2)
('2010-11-14', 1)
('2006-12-26', 1)
('2000-06-20', 2)
('2016-02-03', 2)
('2004-07-30', 2)
('2003-05-06', 1)
('2000-09-10', 1)
('2016-04-10', 1)
('2007-10-10', 1)
('2019-12-22', 1)
('2016-02-25', 1)
('2004-06-21', 2)
('2011-11-26', 1)
('2020-05-05', 1)
('2007-06-07', 1)
('2016-07-27', 2)
('2005-08-01', 1)
('2014-11-15', 2)
('2006-02-16', 2)
('2018-11-06', 1)
('2004-07-12', 1)
('2012-03-22', 1)
('2016-03-12', 2)
('2009-08-31', 1)
('2017-01-09', 1)
('2007-12-30', 1)
('2000-01-17', 1)
('2008-11-17', 2)
('2001-06-09', 1)
('2017-12-22', 2)
('2007-07-27', 1)
('2000-12-

In [57]:
!cat /content/customer_by_state.csv-00000-of-00001.csv


('Unknown', 2357)
